In [912]:
#PATH_data = f"../abyss_data/"
#Mount external drive: sudo mount -t drvfs D: /mnt/d
PATH_data = f"/mnt/d/abyss_data/"
PATH_GENO = f"{PATH_data}genotype"
PATH_SUMSTAT = f"{PATH_data}sumstat/mono/"
PATH_SUMSTAT_no_corr = f"{PATH_data}sumstat_no_corr"
PATH_GENO_segmented = f"{PATH_GENO}/segmented/"
PATH_GENO_mini_dim = f"{PATH_GENO}/mini_dim/"
PATH_GENO_maf = f"{PATH_GENO}/maf/"

PATH_PHENO = f"{PATH_data}phenotype/"
PATH_PHENO_ancestry = f"{PATH_PHENO}/ancestry/all.panel"
PATH_PLOTS = f"{PATH_data}plots/"
PATH_PARAMS = './hyper_params_to_test_AF.txt'

In [913]:
# Import libraries
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import numpy as np
import pandas as pd  # Import Pandas and Numpy to create databases
from collections import Counter
import os
from os import listdir
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4'

In [914]:
effect = 1
chrom = 21
maf_causal_snp = 0.001
sit = "no_strat_causal"

In [915]:
path_genos = PATH_GENO_segmented + f"chrom_{chrom}/unshuffled/"


In [916]:
onlyfiles_input = [f for f in listdir(
    path_genos) if maf_causal_snp + maf_causal_snp/5 > float(f.split('_')[5]) > maf_causal_snp - maf_causal_snp/5]


OSError: [Errno 19] No such device: '/mnt/d/abyss_data/genotype/segmented/chrom_21/unshuffled/'

In [ ]:
popped = onlyfiles_input[round(len(onlyfiles_input)/2)]
snp_nr = 0

In [ ]:
popped

'20_0_1200_20_maf_0.21_0.26.pkl'

In [ ]:
os.system(f"mkdir {PATH_SUMSTAT}maf_causal_{maf_causal_snp}")

256

mkdir: cannot create directory ‘/mnt/d/abyss_data/sumstat/mono/maf_causal_0.25’: File exists


In [ ]:
os.system(f"mkdir {PATH_SUMSTAT}maf_causal_{maf_causal_snp}/{chrom}")


0

In [ ]:
PATH_mono = f"{PATH_SUMSTAT}maf_causal_{maf_causal_snp}"


In [ ]:
try:
  causal_file = [f for f in listdir(
      PATH_mono) if f.startswith("from_chrom")][0]
  snps = pd.read_pickle(f"{PATH_mono}/{causal_file}")
except Exception as e:
  print(e)
  snps = pd.read_pickle(path_genos+popped)
  snps.to_pickle(f"{PATH_mono}/from_chrom_{chrom}_{popped}")
  snp = snps.iloc[:, snp_nr]
  name = snps.columns[snp_nr]
  snp.to_pickle(f"{PATH_mono}/causal_snp_{name}.pkl")

In [ ]:
causal_file = [f for f in listdir(PATH_mono) if f.startswith("from_chrom")][0]


In [ ]:
humans = pd.read_pickle(f"{PATH_data}sumstat/humans_no_causal.pkl")


In [ ]:
snps = pd.read_pickle(f"{PATH_mono}/{causal_file}")

name = snps.columns[snp_nr]

snp_effect = snps.iloc[:, snp_nr] * effect
#snp_effect = snps.iloc[:, snp_nr] * (effect/float(maf.split('_')[0]))

humans['no_strat_causal'] = humans['no_strat_no_causal']
humans['strat_causal'] = humans['strat_no_causal']

humans['no_strat_causal'] = humans['no_strat_causal'] + snp_effect
humans['strat_causal'] = humans['strat_causal'] + snp_effect

In [ ]:
print(Counter(snp_effect).keys())

dict_keys([-1, 1, 0])


In [ ]:
os.system(f"mkdir {PATH_SUMSTAT}maf_causal_{maf_causal_snp}/{chrom}/{sit}")


0

In [ ]:
path_output = f"{PATH_SUMSTAT}maf_causal_{maf_causal_snp}/{chrom}/{sit}/"


In [ ]:
pheno = np.array(humans[sit])

In [ ]:
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

In [ ]:
onlyfiles_input = [f for f in listdir(path_genos)]

In [ ]:
"""
for file in onlyfiles_input:
    path_file = path_genos + '/' + file
    genos = pd.read_pickle(path_file)    
    X = genos
    y = pheno
    X2 = sm.add_constant(X)
    est = sm.OLS(y, X2)
    est2 = est.fit()
    sumstat = pd.DataFrame(data={
        'snp': list(est2.pvalues.index)[1:],
        'coef': list(est2.params)[1:],
        'std_err': list(est2.bse)[1:],
        't': list(est2.tvalues)[1:],
        'P': list(est2.pvalues[1:]),
        '0.025': list(est2.conf_int()[0])[1:],
        '0.975': list(est2.conf_int()[1])[1:]
    })
    sumstat['#CHROM'] = sumstat['snp'].str[:2].astype(int)
    sumstat['POS'] = [i.split(':', 1)[0]
                      for i in list(sumstat['snp'].str[3:-5])]
    sumstat['POS'] = sumstat['POS'].astype(int)
    sumstat['-logp'] = - np.log10(sumstat['P'])
    sumstat.to_pickle(f"{path_output}/{file}")
"""


'\nfor file in onlyfiles_input:\n    path_file = path_genos + \'/\' + file\n    genos = pd.read_pickle(path_file)    \n    X = genos\n    y = pheno\n    X2 = sm.add_constant(X)\n    est = sm.OLS(y, X2)\n    est2 = est.fit()\n    sumstat = pd.DataFrame(data={\n        \'snp\': list(est2.pvalues.index)[1:],\n        \'coef\': list(est2.params)[1:],\n        \'std_err\': list(est2.bse)[1:],\n        \'t\': list(est2.tvalues)[1:],\n        \'P\': list(est2.pvalues[1:]),\n        \'0.025\': list(est2.conf_int()[0])[1:],\n        \'0.975\': list(est2.conf_int()[1])[1:]\n    })\n    sumstat[\'#CHROM\'] = sumstat[\'snp\'].str[:2].astype(int)\n    sumstat[\'POS\'] = [i.split(\':\', 1)[0]\n                      for i in list(sumstat[\'snp\'].str[3:-5])]\n    sumstat[\'POS\'] = sumstat[\'POS\'].astype(int)\n    sumstat[\'-logp\'] = - np.log10(sumstat[\'P\'])\n    sumstat.to_pickle(f"{path_output}/{file}")\n'

In [ ]:
for file in onlyfiles_input:
    path_file = path_genos + '/' + file
    genos = pd.read_pickle(path_file)
    snps_names = []
    coefs = []
    std_errs = []
    ts = []
    ps = []
    twenys = []
    ninetys = []
    for snp_name in genos:
      X = genos[snp_name]
      X2 = sm.add_constant(X)
      y = pheno
      est = sm.OLS(y, X2)
      est2 = est.fit()
      y = pheno
      X2 = sm.add_constant(X)
      est = sm.OLS(y, X2)
      est2 = est.fit()
      snps_names.append(list(est2.pvalues.index)[1:][0])
      coefs.append(list(est2.params)[1:][0])
      std_errs.append(list(est2.bse)[1:][0])
      ts.append(list(est2.tvalues)[1:][0])
      ps.append(list(est2.pvalues[1:])[0])
      twenys.append(list(est2.conf_int()[0])[1:][0])
      ninetys.append(list(est2.conf_int()[1])[1:][0])
    sumstat = pd.DataFrame(data={ 
        'snp': snps_names,
        'coef': coefs,
        'std_err': std_errs,
        't': ts,
        'P': ps,
        '0.025': twenys,
        '0.975': ninetys
    })
    sumstat['#CHROM'] = sumstat['snp'].str[:2].astype(int)
    sumstat['POS'] = [i.split(':', 1)[0]
                      for i in list(sumstat['snp'].str[3:-5])]
    sumstat['POS'] = sumstat['POS'].astype(int)
    sumstat['-logp'] = - np.log10(sumstat['P'])
    sumstat.to_pickle(f"{path_output}/{file}")


In [ ]:
path_sumstat = f"{PATH_SUMSTAT}maf_causal_{maf_causal_snp}/"

In [ ]:
chrom_files = [f for f in listdir(path_sumstat) if f != causal_file]
chrom_files = [f for f in chrom_files if f != f"causal_snp_{name}.pkl"]
chrom_files = [f for f in chrom_files if f.split("_")[0] != f"GWAS"]

In [ ]:
chrom_files

['21', '22']

In [ ]:
sumstat_files

['3_0_1198_3_maf_0.0006_0.001.pkl',
 '3_1_889_3_maf_0.0006_0.001.pkl',
 '3_2_1039_3_maf_0.0006_0.001.pkl',
 '3_3_1039_3_maf_0.0006_0.001.pkl',
 '3_4_1071_3_maf_0.0006_0.001.pkl',
 '3_5_800_3_maf_0.0006_0.001.pkl',
 '3_6_800_3_maf_0.0006_0.001.pkl',
 '3_7_1139_3_maf_0.0006_0.001.pkl',
 '3_8_931_3_maf_0.0006_0.001.pkl',
 '3_9_1094_3_maf_0.0006_0.001.pkl',
 '17_0_1200_17_maf_0.12_0.16.pkl',
 '17_1_1200_17_maf_0.12_0.16.pkl',
 '17_2_1147_17_maf_0.12_0.16.pkl',
 '17_3_800_17_maf_0.12_0.16.pkl',
 '17_4_1174_17_maf_0.12_0.16.pkl',
 '17_5_879_17_maf_0.12_0.16.pkl',
 '17_6_800_17_maf_0.12_0.16.pkl',
 '17_7_1200_17_maf_0.12_0.16.pkl',
 '17_8_800_17_maf_0.12_0.16.pkl',
 '17_9_800_17_maf_0.12_0.16.pkl',
 '15_0_1138_15_maf_0.067_0.093.pkl',
 '15_1_1177_15_maf_0.067_0.093.pkl',
 '15_2_800_15_maf_0.067_0.093.pkl',
 '15_3_1016_15_maf_0.067_0.093.pkl',
 '15_4_1069_15_maf_0.067_0.093.pkl',
 '15_5_1200_15_maf_0.067_0.093.pkl',
 '15_6_800_15_maf_0.067_0.093.pkl',
 '15_7_800_15_maf_0.067_0.093.pkl',
 '15_8

In [ ]:
sumstat_array = []
for chrom_file in chrom_files:
    sumstat_files = [f for f in listdir(path_sumstat + chrom_file + '/' + sit + "/")]
    # first do it with the first one
    path_file = path_sumstat + chrom_file + '/' + sit + '/' + sumstat_files[0]
    sumstats = pd.read_pickle(path_file)
    for sumstat_file in sumstat_files[1:]:
        path_file = path_sumstat + chrom_file + '/' + sit + '/' + sumstat_file
        temp = pd.read_pickle(path_file)
        sumstats = pd.concat([sumstats,temp])
    sumstat_array.append(sumstats)

sumstats = sumstat_array[0]
for sumstat in sumstat_array[1:]:
    sumstats = pd.concat([sumstats,sumstat]) 

In [ ]:
sumstats = sumstats.reset_index(drop=True)

In [ ]:
running_pos = 0
cumulative_pos = []
for chrom, group_df in sumstats.groupby('#CHROM'):
    cumulative_pos.append(group_df['POS'] + running_pos)
    running_pos += group_df['POS'].max()

sumstats['cumulative_pos'] = pd.concat(cumulative_pos)
sumstats['SNP number'] = sumstats.index

In [ ]:
causal_chrom = causal_file.split("_")[2]

In [ ]:
if sit == 'strat_causal':
  title = f"One causal snp with AF of {maf_causal_snp} on chrom {causal_chrom} with a population dependent phenotype"
elif sit == 'strat_no_causal':
  title = f"No causal snp but with a population dependent phenotype"
elif sit == "no_strat_causal":
  title = f"One causal snp with AF of {maf_causal_snp} on chrom {causal_chrom} with no population dependent phenotype"
else:
  title = f"No causal snp and no population dependent phenotype"
sit

'no_strat_causal'

In [ ]:
title

'One causal snp with AF of 0.25 on chrom 21 with no population dependent phenotype'

In [ ]:
sumstats

,snp,coef,std_err,t,P,0.025,0.975,#CHROM,POS,-logp,cumulative_pos,SNP number
0,21:10892322:A:G_A,-1.934729,1.532718,-1.262286,0.206964,-4.940255,1.070798,21,10892322,0.684106,10892322,0
1,21:47368531:A:G_G,1.141090,1.533036,0.744333,0.456745,-1.865061,4.147240,21,47368531,0.340326,47368531,1
2,21:32892189:A:G_G,-2.319273,1.769433,-1.310744,0.190065,-5.788976,1.150430,21,32892189,0.721099,32892189,2
3,21:15792338:A:C_C,0.632022,1.533154,0.412236,0.680202,-2.374360,3.638403,21,15792338,0.167362,15792338,3
4,21:25645657:G:T_T,-1.948166,1.532711,-1.271059,0.203826,-4.953679,1.057347,21,25645657,0.690741,25645657,4
...,...,...,...,...,...,...,...,...,...,...,...,...
479995,22:31587218:C:T_C,-0.085368,0.117760,-0.724933,0.468561,-0.316284,0.145549,22,31587218,0.329234,79706852,479995
479996,22:37377737:C:T_T,-0.014694,0.126099,-0.116527,0.907245,-0.261963,0.232575,22,37377737,0.042276,85497371,479996
479997,22:30678530:A:G_A,0.005425,0.119980,0.045212,0.963942,-0.229846,0.240695,22,30678530,0.015949,78798164,479997
479998,22:27063804:C:G_G,0.175412,0.126437,1.387339,0.165462,-0.072521,0.423344,22,27063804,0.781302,75183438,479998


In [ ]:
sumstats['causality'] = np.zeros(sumstats.shape[0])

In [ ]:
sumstats['causality'].mask(sumstats['snp'] == name, 1, inplace=True)


In [ ]:
sumstats = sumstats.sort_values(by=['causality'])


In [ ]:
sns.relplot(
    data=sumstats,
    x='cumulative_pos',
    y='-logp',
    aspect=2,
    hue='#CHROM',
    style = "causality",
    palette='dark'
).set(title=title)
plt.savefig(f"{path_sumstat}/GWAS_mono_{sit}.png")
plt.savefig(f"./GWAS_Common.png")